In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import json

In [3]:
tree_df = pd.read_csv('./final_data/trees_sample.csv')
borough_df = pd.read_csv('./final_data/boroughs_sample.csv')
neighborhood_df = pd.read_csv('./final_data/neighborhoods_sample.csv')

In [9]:

with open('./geo_data/borough.geo.json', 'r') as file:
    boroughs = json.load(file)
with open('./geo_data/UHF34.geo.json', 'r') as file:
    nbds = json.load(file)
borough_df['BoroName'] = borough_df['Borough']
boroughs["features"][0]

{'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-73.89680883223774, 40.79580844515977],
     [-73.89796839783742, 40.79564483916199],
     [-73.89919434249983, 40.79650245601821],
     [-73.89788253240185, 40.79711653214703],
     [-73.89713149795642, 40.79679807772829],
     [-73.89680883223774, 40.79580844515977]]],
   [[[-73.87287195837754, 40.785975027804014],
     [-73.87288496983653, 40.785962633215966],
     [-73.87351452458638, 40.78575009169909],
     [-73.87532222427274, 40.78587448571043],
     [-73.876772248509, 40.7858004589828],
     [-73.8783068005765, 40.785356620508445],
     [-73.88596200181902, 40.78675251948481],
     [-73.88692199720253, 40.78673311425362],
     [-73.88830725732264, 40.78700634478622],
     [-73.88940460807085, 40.787791280074515],
     [-73.89012684970685, 40.78963627999305],
     [-73.89056971575059, 40.790107636212866],
     [-73.89218182984939, 40.79039426960896],
     [-73.89282283610936, 40.79281708195142],
    

In [5]:
# TODO: make the map ???
fig = go.Figure(data=go.Scattergeo(
        lon = tree_df['Long'],
        lat = tree_df['Lat'],
        text = tree_df['Diameter'],
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'blue'
        )
        ))

fig.update_layout(geo = dict(
        landcolor = "rgb(212, 212, 212)",
        resolution = 50,
        lonaxis = dict(
            showgrid = False,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = False,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    )
map = go.Figure(data=fig)
fig.show()